#CEM300-Natural-Language-Processing-Coursework

In this work, I explore the Product Classification and Clustering Dataset.

<br>
Sources:

Dataset Chosen - https://archive.ics.uci.edu/dataset/837/product+classification+and+clustering

<br>




The first task is to implement